<a href="https://colab.research.google.com/github/PavankumarUppar/DLL/blob/main/Program8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

In [3]:
def generate_synthetic_data(num_samples=1000, max_sequence_length=20, embedding_dim=50):
    np.random.seed(42)
    X = np.random.randint(0, 2, size=(num_samples, max_sequence_length))
    y = np.random.randint(0, 2, size=(num_samples,))
    return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.long)

In [4]:
class SentimentLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, embedding_dim):
        super(SentimentLSTM, self).__init__()
        self.embedding = nn.Embedding(input_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)
        lstm_out = lstm_out[:, -1, :]
        output = self.fc(lstm_out)
        output = self.sigmoid(output)
        return output

In [18]:
input_size = 200
hidden_size = 64
num_layers = 2
output_size = 1
embedding_dim = 50

In [19]:
model = SentimentLSTM(input_size, hidden_size, num_layers, output_size, embedding_dim)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [21]:
X, y = generate_synthetic_data(num_samples=1000, max_sequence_length=20, embedding_dim=embedding_dim)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [24]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs.long())
        loss = criterion(outputs.squeeze(), labels.float())
        loss.backward()
        optimizer.step()

In [25]:
model.eval()
all_predictions = []
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs.long())
        predictions = (outputs.squeeze() > 0.5).long()
        all_predictions.extend(predictions.tolist())

In [26]:
accuracy = accuracy_score(y_test.numpy(), all_predictions)
print(f"Accuracy on test set: {accuracy * 100:.2f}%")

Accuracy on test set: 50.00%


In [27]:
def preprocess_text_input(text, max_sequence_length=20):
    tokens = [ord(char) for char in text]


    if len(tokens) < max_sequence_length:
        tokens += [0] * (max_sequence_length - len(tokens))
    else:
        tokens = tokens[:max_sequence_length]


    tensor_input = torch.tensor(tokens, dtype=torch.long)

    return tensor_input


custom_input_text = "This is a positive example."


custom_input_tensor = preprocess_text_input(custom_input_text)


model.eval()


print(f"Token Indices: {custom_input_tensor}")


with torch.no_grad():
    try:
        output = model(custom_input_tensor.unsqueeze(0))
        predicted_sentiment = "Positive" if output.item() > 0.5 else "Negative"
        print(f"Predicted sentiment: {predicted_sentiment} (Probability: {output.item():.4f})")
    except IndexError as e:
        print(f"Error: {e}")
        print("Check if the token indices in the custom input are within the specified vocabulary size.")


Token Indices: tensor([ 84, 104, 105, 115,  32, 105, 115,  32,  97,  32, 112, 111, 115, 105,
        116, 105, 118, 101,  32, 101])
Predicted sentiment: Positive (Probability: 0.5128)
